In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
#import de la base
df = pd.read_csv("databases/bpe19_ensemble_xy.csv", sep =";")

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [78]:
df['gps_null'] = df['LAMBERT_X'].isna().astype('str')
df['gps_null'].dtype

dtype('O')

In [84]:
count = df.groupby(['TYPEQU', 'gps_null'])['REG'].count().reset_index().rename(columns = {'REG': 'nb_eq'})
count.head(20)

,TYPEQU,gps_null,nb_eq
0,A101,False,777
1,A101,True,1
2,A104,False,3014
3,A104,True,6
4,A105,False,34
5,A105,True,1
6,A106,False,168
7,A106,True,1
8,A107,False,283
9,A107,True,1


In [85]:
count[count['TYPEQU'] == 'D107'].head(50)

,TYPEQU,gps_null,nb_eq
171,D107,False,482
172,D107,True,28


In [86]:
count = count.pivot(index = 'TYPEQU', columns = 'gps_null', values = 'nb_eq').reset_index()
count.head()

gps_null,TYPEQU,False,True
0,A101,777.0,1.0
1,A104,3014.0,6.0
2,A105,34.0,1.0
3,A106,168.0,1.0
4,A107,283.0,1.0


In [87]:
count = count.fillna(0)

In [89]:
count['nb_eq'] = count['False'] + count['True']
count['pct_null'] = count['True'] / count['nb_eq']
count = count.sort_values(by = "pct_null", ascending = False)
count.head()

gps_null,TYPEQU,False,True,nb_eq,pct_null
26,A401,0.0,126931.0,126931.0,1.000000
43,B202,30061.0,2584.0,32645.0,0.079155
97,D107,482.0,28.0,510.0,0.054902
88,C609,1424.0,77.0,1501.0,0.051299
86,C604,98.0,4.0,102.0,0.039216


In [111]:
equip = pd.read_csv('databases/code_equipement.csv', sep = ";", encoding = 'utf8')
equip = equip.rename(columns = {'Code': 'TYPEQU'})
equip.head()

,TYPEQU,Equipement
0,A101,Police
1,A104,Gendarmerie
2,A105,Cour d’appel (CA)
3,A106,Tribunal de grande instance (TGI)
4,A107,Tribunal d’instance (TI)


In [119]:
count = count.merge(equip, on = "TYPEQU", how = "left")

In [120]:
count.to_csv('pct_null_per_typequ.csv', encoding = 'utf8')